In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
! wget https://storage.googleapis.com/class25jan2022/share/2008.csv

--2022-02-12 23:44:41--  https://storage.googleapis.com/class25jan2022/share/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 74.125.199.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M   235MB/s    in 2.8s    

2022-02-12 23:44:44 (235 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [8]:
! wc -l ./2008.csv

7009729 ./2008.csv


In [9]:
! head -3 2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


In [10]:
! hdfs dfs -rm -f /user/cloudera/input/2008.csv

/bin/bash: hdfs: command not found


In [11]:
! hdfs dfs -mkdir -p /user/cloudera/input

/bin/bash: hdfs: command not found


In [12]:
! hadoop fs -put 2008.csv /user/cloudera/input

/bin/bash: hadoop: command not found


In [13]:
! hdfs dfs -ls /user/cloudera/input

/bin/bash: hdfs: command not found


In [14]:
file_path = '2008.csv'

In [15]:
airline_df = spark.read.format('csv').option('header', 'true').option('mode', 'DROPMALFORMED').load(file_path)

In [16]:
airline_df.count()

7009728

In [17]:
airline_row_df = airline_df

In [18]:
airline_row_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [19]:
from pyspark.sql.types import *
from pyspark.sql import functions as sparkf

In [20]:
crunched_df = airline_row_df.\
withColumn('DepTime', sparkf.col('DepTime').cast(DoubleType())).\
withColumn('TaxiOut', sparkf.col('TaxiOut').cast(DoubleType())).\
withColumn('TaxiIn', sparkf.col('TaxiIn').cast(DoubleType())).\
withColumn('DepDelay', sparkf.col('DepDelay').cast(DoubleType())).\
withColumn('DayOfWeek', sparkf.col('DayOfWeek').cast(DoubleType())).\
withColumn('Distance', sparkf.col('Distance').cast(DoubleType())).\
withColumn('ArrDelay', sparkf.col('ArrDelay').cast(DoubleType()))

In [21]:
crunched_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: double (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [22]:
! pip install pandas

In [23]:
import pandas as pd
crunched_df.select(['DepTime', 'TaxiOut', 'TaxiIn', 'DayOfWeek', 'Distance', 'ArrDelay']).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
DepTime,6873482,1333.8300461105448,478.06889486629836,1.0,2400.0
TaxiOut,6872670,16.453045177492882,11.332798654232155,0.0,429.0
TaxiIn,6858079,6.860851704974527,4.933649371300466,0.0,308.0
DayOfWeek,7009728,3.9241815088973495,1.9882589459851212,1.0,7.0
Distance,7009728,726.3870294253928,562.1018034840403,11.0,4962.0
ArrDelay,6855029,8.16845238729114,38.50193694882867,-519.0,2461.0


In [24]:
def t_timeperiod(origin):
  if origin is None:
    period = None
  elif origin > 0 and origin < 600:
    period = '00.01-05.59'
  elif origin >= 600 and origin < 1200:
    period = '06.01-11.59'
  elif origin >= 1200 and origin < 1800:
    period = '12.00-17.59'
  elif origin >= 1800 and origin <= 2400:
    period = '18.00-24.00'
  else:
    period = 'NA'
  return period

In [25]:
timeperiod = sparkf.udf(lambda x: t_timeperiod(x), StringType())

In [26]:
discretized_df = crunched_df.withColumn('DepTime', timeperiod(sparkf.col('DepTime')))

In [27]:
max_distance = discretized_df.select(sparkf.max('Distance')).collect()[0][0]
min_distance = discretized_df.select(sparkf.min('Distance')).collect()[0][0]

In [28]:
max_ArrDelay = discretized_df.select(sparkf.max('ArrDelay')).collect()[0][0]
min_ArrDelay = discretized_df.select(sparkf.min('ArrDelay')).collect()[0][0]

In [29]:
def t_normalized_distance(origin):
  if origin is None:
    return None
  else:
    return((origin - min_distance) / (max_ArrDelay - min_distance))

In [30]:
def t_normalized_ArrDelay(origin):
  if origin is None:
    return None
  else:
    return((origin - min_ArrDelay) / (max_ArrDelay - min_ArrDelay))

In [31]:
normalized_distance = sparkf.udf(lambda x: t_normalized_distance(x), DoubleType())

In [32]:
normalized_ArrDelay = sparkf.udf(lambda x: t_normalized_ArrDelay(x), DoubleType())

In [33]:
normalized_df = discretized_df.\
withColumn('Distance', normalized_distance(sparkf.col('Distance'))).\
withColumn('ArrDelay', normalized_distance(sparkf.col('ArrDelay')))

In [34]:
features_df = normalized_df.\
select(['UniqueCarrier', 'Origin', 'Dest',\
        'DepTime', 'TaxiOut', 'TaxiIn', 'DepDelay',\
        'DayOfWeek', 'Distance', 'ArrDelay'])

In [35]:
final_df = features_df.dropna()

In [36]:
final_df.count()

6855029

In [37]:
final_df.show()

+-------------+------+----+-----------+-------+------+--------+---------+--------------------+--------------------+
|UniqueCarrier|Origin|Dest|    DepTime|TaxiOut|TaxiIn|DepDelay|DayOfWeek|            Distance|            ArrDelay|
+-------------+------+----+-----------+-------+------+--------+---------+--------------------+--------------------+
|           WN|   IAD| TPA|18.00-24.00|    8.0|   4.0|     8.0|      4.0| 0.32612244897959186|-0.01020408163265306|
|           WN|   IAD| TPA|06.01-11.59|   10.0|   5.0|    19.0|      4.0| 0.32612244897959186|-0.00367346938775...|
|           WN|   IND| BWI|06.01-11.59|   17.0|   3.0|     8.0|      4.0|  0.2057142857142857|0.001224489795918...|
|           WN|   IND| BWI|06.01-11.59|    7.0|   3.0|    -4.0|      4.0|  0.2057142857142857|-0.00693877551020...|
|           WN|   IND| BWI|18.00-24.00|   10.0|   3.0|    34.0|      4.0|  0.2057142857142857|0.009387755102040816|
|           WN|   IND| JAX|18.00-24.00|   10.0|   4.0|    25.0|      4.0

In [74]:
training_df, test_df = final_df.randomSplit([0.8,0.2], seed=12)

In [75]:
training_df.count()

5484163

In [76]:
training_df.printSchema()

root
 |-- UniqueCarrier: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- DayOfWeek: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- ArrDelay: double (nullable = true)



In [77]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [78]:
DepTimeIndexer = StringIndexer\
(handleInvalid = 'keep', inputCol = 'DepTime', outputCol = 'DepTimeIndexed')

In [79]:
UniqueCarrierIndexer = StringIndexer\
(handleInvalid = 'keep', inputCol = 'UniqueCarrier', outputCol = 'UniqueCarrierIndexed')

In [80]:
UniqueCarrierOneHotEncoder = OneHotEncoder\
(dropLast = False, inputCol = 'UniqueCarrierIndexed',outputCol = 'UniqueCarrierVec')

In [81]:
OriginIndexer = StringIndexer(handleInvalid = 'keep', inputCol = 'Origin', outputCol = 'OriginIndexed')

In [82]:
OriginOneHotEncoder = OneHotEncoder\
(dropLast = False, inputCol = 'OriginIndexed',outputCol = 'OriginVec')

In [83]:
DestIndexer = StringIndexer\
(handleInvalid = 'keep', inputCol = 'Dest', outputCol = 'DestIndexed')

In [84]:
DestOneHotEncoder = OneHotEncoder\
(dropLast = False, inputCol = 'DestIndexed',outputCol = 'DestVec')

In [85]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [86]:
featureAssembler = VectorAssembler\
(inputCols = ['DepTimeIndexed',\
              'UniqueCarrierVec',\
              'OriginVec',\
              'DestVec',\
              'TaxiOut', 'TaxiIn',\
              'DepDelay',\
              'DayOfWeek',\
              'Distance'
             ], outputCol = '***features')

In [97]:
from pyspark.ml.regression import GeneralizedLinearRegression

In [98]:
dt = GeneralizedLinearRegression\
(labelCol = 'ArrDelay', featuresCol = '***features')

In [99]:
Pipeline

pyspark.ml.pipeline.Pipeline

In [100]:
pipeline_dt = Pipeline().\
setStages([DepTimeIndexer,\
           UniqueCarrierIndexer,\
           UniqueCarrierOneHotEncoder,\
           OriginIndexer,\
           OriginOneHotEncoder,\
           DestIndexer,\
           DestOneHotEncoder,\
           featureAssembler, dt])

In [101]:
dtModel = pipeline_dt.fit(training_df)

In [102]:
result_df = dtModel.transform(test_df.dropna())

In [103]:
result_df.select(['ArrDelay', 'Prediction']).show()

+--------------------+--------------------+
|            ArrDelay|          Prediction|
+--------------------+--------------------+
|-0.00530612244897...|-0.00767538206521...|
|0.003265306122448...|0.001497384849399203|
|-0.00938775510204...|-0.00619083926519...|
|-0.00775510204081...|-0.00594178010248...|
|-0.00163265306122...|-0.00128154652115...|
|-0.00244897959183...|-0.00136184224941...|
|0.008979591836734694|  0.0117447074788285|
|-0.00408163265306...|-0.00473581461922...|
|-0.00571428571428...|-0.00556507038272552|
|-0.01020408163265306|-0.01088154683761...|
|-0.00775510204081...|-0.00970368235023887|
|-8.16326530612244...|-0.00666231036442...|
|-0.01306122448979...|-0.00996988402131...|
|-0.01020408163265306|-0.01041510193410...|
|0.011020408163265306|0.007411736938504504|
|-0.00489795918367...|-0.00244241934961...|
|0.001224489795918...|-1.58481841255416...|
|-4.08163265306122...|-0.00174355869030...|
|-0.00163265306122...|-0.00989463279454...|
|-0.01061224489795...|-0.0091025

In [104]:
from pyspark.ml.evaluation import RegressionEvaluator

In [105]:
lr_evaluator_r2 = RegressionEvaluator\
(predictionCol = "prediction", labelCol = "ArrDelay", metricName = "r2")
print("R Squared (R2) on test data = %g"\
      % lr_evaluator_r2.evaluate(result_df))

R Squared (R2) on test data = 0.944345


In [106]:
lr_evaluator_rmse = RegressionEvaluator\
(predictionCol = "prediction", labelCol = "ArrDelay", metricName = "rmse")
print("R Squared (R2) on test data = %g"\
      % lr_evaluator_rmse.evaluate(result_df))

R Squared (R2) on test data = 0.00371547
